#언급횟수

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
import pandas as pd
contents = pd.read_csv('/content/gdrive/MyDrive/ITStudy/크롤링/contents.csv')
contents['content'] = contents['title'] + contents['contents']
contents = contents[['date','id','content']]
contents['reply'] = ''
reply = pd.read_csv('/content/gdrive/MyDrive/ITStudy/크롤링/reply.csv',encoding='utf-8')
keyword = pd.read_csv('/content/gdrive/MyDrive/ITStudy/크롤링/keyword.txt',encoding='utf-8',header=None)

In [ ]:
# 확인용
contents.head()

,date,id,content,reply
0,8.23,308999,우리카드 추천해주실분계신가요?\n실적없고 그냥 국내 할인혜택 한번씩 쓸까해서 체...,
1,8.23,308946,"아 진짜 너무 고민중인데 카드 한 장만 추천해줘라\n매 월 꿀카드, 톡마포, 더 나...",
2,8.23,308883,알뜰 교통카드에 관해서 문의 좀....!!\n0. 현재예전에 카드사에서 추천해줘서내...,
3,8.23,308843,국민에서 카드 추천 연락 왔는데\n조건 충족해서 전화 온건가…? 내가 소득 없는 학...,
4,8.23,308815,조언 부탁드립니다!\n제가 1년에 한번씩은 꼭 해외여행을 가서 마일리지 카드를쓰기도...,


In [ ]:
keyword.head()

,0
0,우리카드
1,트래블로그
2,viva
3,와우리
4,우리카드정석


In [ ]:
# 중복제거
keyword = keyword.drop_duplicates(subset=None, keep='first', inplace=False, ignore_index=False)
keyword

,0
0,우리카드
1,트래블로그
2,viva
3,와우리
4,우리카드정석
...,...
343,레드스프라이트
346,댄공카드
347,판타스틱s
348,삼성 국민행복


In [ ]:
for idx, row in contents.iterrows():
    reply_arr = ""
    for didx, drow in reply.iterrows():
        if row.id == drow.id:
            reply_arr += drow.reply_content
    contents.at[idx, 'reply'] = reply_arr

In [ ]:
# 필요한 것들 설치, import

! pip install tensorflow
! pip install keras
# 띄어쓰기 처리
!pip install git+https://github.com/haven-jeon/PyKoSpacing.git --no-deps argument
! pip3 install konlpy

import numpy as np
import pandas as pd


  Cloning https://github.com/haven-jeon/PyKoSpacing.git to /tmp/pip-req-build-3_xuhr6l
  Running command git clone --filter=blob:none --quiet https://github.com/haven-jeon/PyKoSpacing.git /tmp/pip-req-build-3_xuhr6l
  Resolved https://github.com/haven-jeon/PyKoSpacing.git to commit d1e2f93759b1bcc74b6aaa345a62c08701e5546e
  Preparing metadata (setup.py) ... done


In [ ]:
# 불용어 제거

import re

def clean_str(text):
    # pattern = '([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)'
    # text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+'
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '([ㄱ-ㅎㅏ-ㅣ]+)'
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '<[^>]*>'
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '[^\w\s\n]'
    text = re.sub(pattern=pattern, repl='', string=text)
    text = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]','', string=text)
    text = re.sub('\n', '.', string=text)

    return text

In [ ]:
for idx, row in contents.iterrows():
    clean_content = clean_str(row.content)
    clean_reply = clean_str(row.reply)
    contents.at[idx,'content'] = clean_content
    contents.at[idx, 'reply'] = clean_reply

In [ ]:
contents.head()

,date,id,content,reply
0,8.23,308999,우리카드 추천해주실분계신가요.실적없고 그냥 국내 할인혜택 한번씩 쓸까해서 체크도...,실적 없는거면 카드의정석 에브리원 어떰 dc App정보보겠씸 신카 실적없는거 찾...
1,8.23,308946,아 진짜 너무 고민중인데 카드 한 장만 추천해줘라.매 월 꿀카드 톡마포 더 나은 나...,매달 스포티비 때문에 네이버페이 쓰는데 신한 네이버페이 알아볼까상품권 6 전환한거...
2,8.23,308883,알뜰 교통카드에 관해서 문의 좀.0 현재예전에 카드사에서 추천해줘서내가 지금 국민 ...,알뜰교통에 신용은 연회비가 아까운데 체크로 실사할려면 우리 체크가 좋고 실사는 하...
3,8.23,308843,국민에서 카드 추천 연락 왔는데.조건 충족해서 전화 온건가 내가 소득 없는 학식이라...,마케팅 동의 철회
4,8.23,308815,조언 부탁드립니다.제가 1년에 한번씩은 꼭 해외여행을 가서 마일리지 카드를쓰기도 하...,마일리지 카드로 상테크 돌리고 평소엔 할인적립카드 쓰셈네 이게 제일 좋겠네요 감사합...


In [ ]:
tf_contents = contents[['content']]
tf_contents

,content
0,우리카드 추천해주실분계신가요.실적없고 그냥 국내 할인혜택 한번씩 쓸까해서 체크도...
1,아 진짜 너무 고민중인데 카드 한 장만 추천해줘라.매 월 꿀카드 톡마포 더 나은 나...
2,알뜰 교통카드에 관해서 문의 좀.0 현재예전에 카드사에서 추천해줘서내가 지금 국민 ...
3,국민에서 카드 추천 연락 왔는데.조건 충족해서 전화 온건가 내가 소득 없는 학식이라...
4,조언 부탁드립니다.제가 1년에 한번씩은 꼭 해외여행을 가서 마일리지 카드를쓰기도 하...
...,...
323,병원비결제 카드 추천.병원비 결제할 일이 있이 생겼는데 5001000 정도내가 미리...
324,월 150쓰는데 카드추천좀.원큐쿠폰어쩌고 카드 매달 150정도 카드값나감혜택 뭐가 ...
325,실사용으로 560쓰는데 신용카드 추천좀.현재 통신사30에 12 할인 톡마포 나사카...
326,대학생 카드 추천좀.전역하고 군적금 다 부모님 드림 소득 0원이고 한달마다 용돈 받...


In [ ]:
tf_reply = contents[['reply']]
tf_reply

,reply
0,실적 없는거면 카드의정석 에브리원 어떰 dc App정보보겠씸 신카 실적없는거 찾...
1,매달 스포티비 때문에 네이버페이 쓰는데 신한 네이버페이 알아볼까상품권 6 전환한거...
2,알뜰교통에 신용은 연회비가 아까운데 체크로 실사할려면 우리 체크가 좋고 실사는 하...
3,마케팅 동의 철회
4,마일리지 카드로 상테크 돌리고 평소엔 할인적립카드 쓰셈네 이게 제일 좋겠네요 감사합...
...,...
323,그니마 카정포 여러개 인데 전월실적 필요하고 에블라도 돈 적게 해줘서 대체체 없음...
324,하포신한 하이포인트하포실적 150채우고 원카드 쓰고 싶으면 하포가 젤 편하고 좋아일...
325,이미 잘쓰고있구만 거기서 뭐 추가할거없음 카정포 30써야 1만원인데 통신사가 2천원...
326,걍 나사카 계속 쓰셈삼성페이 등록 안돼서 불편했는데 걍 써야겠네 dc App삼...


In [ ]:
###문장분석###
##한국어##
from konlpy.tag import Okt
okt = Okt()
result =[]
result = [okt.nouns(str(contents))] #명사만 추출
# final_result= [r for i in result for r in i]
# print(final_result)
print(result)

[['우리카드', '추천', '분계', '신가', '실적', '그냥', '국내', '할인', '혜택', '한번', '체크', '진짜', '고민', '카드', '장만', '추천', '매', '월', '꿀', '카드', '톡', '마포', '더', '나은', '나', '알뜰', '교통카드', '관해', '문의', '좀', '현재', '예전', '카드', '사', '추천', '내', '지금', '국민', '국민', '카드', '추천', '연락', '조건', '충족', '전화', '온건', '내', '소득', '학식', '조언', '제', '한번', '꼭', '해외여행', '마일리지', '카드', '쓰기', '병원', '비', '결제', '카드', '추천', '병원', '비', '결제', '정도', '내', '미리', '월', '카드', '추천', '좀', '큐', '쿠폰', '쩌', '카드', '매달', '정도', '카드', '값', '혜택', '뭐', '실', '사용', '신용카드', '추천', '좀', '현재', '통신사', '할인', '톡', '마포', '나사', '카', '대학생', '카드', '추천', '좀', '전역', '군', '적금', '부모님', '드림', '소득', '달마', '용돈', '초반', '대학생', '신용카드', '추천', '좀', '군', '적금', '해지', '국민', '영업', '하나', '만', '실적', '카드', '정석', '에브리', '정보', '씸', '카', '실적', '매달', '스포', '티비', '때문', '네이버', '페이', '신한', '네이버', '페이', '상품권', '전환', '거', '알뜰', '교통', '신용', '회비', '체크', '실사', '우리', '체크', '실사', '마케팅', '동의', '철회', '마일리지', '카드', '상', '테크', '평소', '할인', '적립', '카드', '이', '제일', '마', '카정포', '개', '전월', '실적', '에블라', '돈', '대체', '체'

In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
stop_word = []
with open("/content/gdrive/MyDrive/ITStudy/크롤링/stopword2.txt", "r") as f:
    for line in f:
        stop_word.append(line.strip())

start_word = []
with open("/content/gdrive/MyDrive/ITStudy/크롤링/keyword.txt", "r") as f:
    for line in f:
        start_word.append(line.strip())

In [ ]:
print(start_word)

['우리카드', '트래블로그', 'viva', '와우리', '우리카드정석', 'da@', '알뜰', '교통카트', '국민', '딥드림', '하포', '그린', 'simple life 하나카드', 'nu우리', '알뜰교통카드', '딥애코', '딥드림플래티넘', '플래티넘', '현대M1', 'M1', '레클', '현카', '언리밋', '알뜰교통', '톡마포', '딥드플', '가온실속', '현대', 'M3부스트', '현대M3부스트', '원더프리', '와우리', '원더프리+', '카정포', '스마트애니', '케티퍼', '현대제로', '로카1.2', 'MX블랙', 'MX', '카카오신한', '마일리지', '현대카드', '삼성 id on', '욜로', '하포', '로얄블루', '베브9', '현아골', '본보이', '헤리티지', '롯데카드', '현아플', '스이카', '파스모', '이코카', '신한카드', '국민카드', '에블라', '딥오일', '국민카드', '네이버페이', '금리포', '가온실속', '레드5', '가온올림', '더모아', '클y', '욜로', '테이스티', '라이킷', '케이퍼스트', '더쎈카드', '꿀카드', '톡마포', '더나은', '나사카', '액부', '제로', '현대', 'm부스트', 'M2', '신한삑카드', '신한플리카드', '미스터라이프', '라인프렌즈', '하이포인트', '퍼플', '네이버 현대', '라인프렌즈 신한', '롯데1.2였나', '무신사 현대', '국민 가온글로벌', '노리카드', 'simple life 하나카드', '딥에코', '국카', '위시올', '언리미티드', '현아플', '삼아플', '퍼플', '에블라', '클래식y', '딥드플', '하포', '와우리', '톡마포', '톡톡 마이 포인트', 'IBK일상의 기쁨', 'IBK', '우리 olleh ceo 카드', '신한 샵페이', '삼성 마일리지 플래티넘', '롯데쿠팡카드', '우리 CGV 카드', '우리DA@카드의 정석', 'toptop s', '전우애 패밀리', '제일 

In [ ]:
content_results = []
reply_results = []

for idx, row in contents.iterrows():
    content_tokens = word_tokenize(row.content)
    reply_tokens = word_tokenize(row.reply)

    content_result = []
    reply_result = []
    for w in content_tokens:
        if w in start_word:
            content_result.append(w)

    for w in reply_tokens:
        if w in start_word:
            reply_result.append(w)

    content_results.append(content_result)
    reply_results.append(reply_result)


In [ ]:
len(content_results)

328

In [ ]:
results = content_results + reply_results
results

[['우리카드', '트래블로그', 'viva', '와우리', '우리카드정석'],
 ['꿀카드', '톡마포'],
 ['알뜰',
  '국민',
  '탄탄대로',
  '알뜰교통',
  '신한카드',
  '알뜰교통',
  '우리카드',
  '탄탄대로',
  '알뜰',
  '탄탄대로',
  '알뜰'],
 [],
 ['마일리지', '마일리지'],
 [],
 ['딥드림', '딥드림'],
 ['그린'],
 ['딥드림', '하포', '현대', '엑부', '제로'],
 ['m부스트', '현대카드', '현대카드'],
 [],
 ['신한카드'],
 [],
 [],
 [],
 [],
 ['네이버페이', '네이버페이', '라인프렌즈', '신한카드', '국민'],
 [],
 [],
 [],
 ['현아플'],
 [],
 [],
 ['현대', '국민', '하이포인트'],
 [],
 ['현대', '국민', '하이포인트'],
 [],
 [],
 ['국민'],
 [],
 [],
 [],
 ['알뜰교통카드', '알뜰', '알뜰', '알뜰'],
 [],
 ['딥애코'],
 ['더모아', '더모아'],
 [],
 [],
 [],
 ['딥드림'],
 ['국민카드', '국카'],
 ['위시올'],
 ['현대M1', '레클'],
 ['케이퍼스트'],
 [],
 ['넥슨', '현카', '언리밋', '현대', '언리미티드', '카정포', '현대', '언리미티드'],
 [],
 [],
 ['현아플', '삼아플', '퍼플', '에블라', '클래식y', '딥드플', '하포', '와우리'],
 [],
 [],
 ['알뜰교통'],
 [],
 [],
 [],
 ['IBK', '현대'],
 ['국민'],
 ['톡마포', '샵페이', '마일리지', '플래티넘', '롯데쿠팡카드'],
 [],
 ['딥드림', '탄탄대로', '탄탄대로', '현대'],
 ['클래식s'],
 [],
 [],
 [],
 ['네이버페이', '신한카드'],
 ['딥드플', '네이버페이', '딥드플'],
 [],
 [],
 ['현대'],
 ['현대'],


In [ ]:
def flatten(arg):
    ret = []
    for i in arg:
        ret.extend(i) if isinstance(i, list) else ret.append(i)
    return ret

word = flatten(results)

# contents_word = flatten(content_results)
# reply_word = flatten(reply_results)

from collections import Counter # 딕셔너리를 잘 사용하기 위한 패키지

counts = Counter(word)
counts
tags = counts.most_common()

In [ ]:
tags = pd.DataFrame(tags, columns= ['카드', '횟수'])
tags

,카드,횟수
0,톡마포,54
1,국민,42
2,딥드림,39
3,현대,37
4,마일리지,28
...,...,...
124,트래블월렛,1
125,노리2,1
126,카정DA,1
127,댄공카드,1


In [ ]:
tags.to_csv("/content/gdrive/MyDrive/ITStudy/크롤링/tags.csv", index=False)

# TF-IDF

In [ ]:
import pandas as pd
contents = pd.read_csv('/content/gdrive/MyDrive/ITStudy/크롤링/contents.csv')
contents['content'] = contents['title'] + contents['contents']
contents = contents[['id','content']]
contents['reply'] = ''
reply = pd.read_csv('/content/gdrive/MyDrive/ITStudy/크롤링/reply.csv',encoding='utf-8')

FileNotFoundError: ignored

In [ ]:
for idx, row in contents.iterrows():
    reply_arr = ""
    for didx, drow in reply.iterrows():
        if row.id == drow.id:
            reply_arr += drow.reply_content
    contents.at[idx, 'reply'] = reply_arr

In [ ]:
contents.head()

,id,content,reply
0,308999,우리카드 추천해주실분계신가요?\n실적없고 그냥 국내 할인혜택 한번씩 쓸까해서 체...,실적 없는거면 카드의정석 에브리원 어떰? - dc App정보보겠씸 ㄳㄳ신카 실적없...
1,308946,"아 진짜 너무 고민중인데 카드 한 장만 추천해줘라\n매 월 꿀카드, 톡마포, 더 나...",+ 매달 스포티비 때문에 네이버페이 쓰는데 신한 네이버페이 알아볼까?상품권 6% 전...
2,308883,알뜰 교통카드에 관해서 문의 좀....!!\n0. 현재예전에 카드사에서 추천해줘서내...,알뜰교통에 신용은 연회비가 아까운데 체크로 ㄱ 실사할려면 우리 체크가 좋고 실사는 ...
3,308843,국민에서 카드 추천 연락 왔는데\n조건 충족해서 전화 온건가…? 내가 소득 없는 학...,마케팅 동의 철회 ㄱ
4,308815,조언 부탁드립니다!\n제가 1년에 한번씩은 꼭 해외여행을 가서 마일리지 카드를쓰기도...,마일리지 카드로 상테크 돌리고 평소엔 할인적립카드 쓰셈네 이게 제일 좋겠네요 감사합...


In [ ]:
# 불용어 제거

import re

def clean_str(text):
    pattern = '([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)'
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+'
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '([ㄱ-ㅎㅏ-ㅣ]+)'
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '<[^>]*>'
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '[^\w\s\n]'
    text = re.sub(pattern=pattern, repl='', string=text)
    text = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]','', string=text)
    text = re.sub('\n', '.', string=text)

    return text

In [ ]:
for idx, row in contents.iterrows():
    clean_content = clean_str(row.content)
    clean_reply = clean_str(row.reply)
    contents.at[idx,'content'] = clean_content
    contents.at[idx, 'reply'] = clean_reply

In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
def tokenize(text):

   # normalize case and remove punctuation
   text = text.split()

   return text

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# initialize count vectorizer object
vect = CountVectorizer(tokenizer = tokenize)

In [ ]:
contents['content'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 328 entries, 0 to 327
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       328 non-null    int64 
 1   content  328 non-null    object
 2   reply    328 non-null    object
dtypes: int64(1), object(2)
memory usage: 7.8+ KB


In [ ]:
X = vect.fit_transform(contents['content'])
len(X.toarray())

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


328

In [ ]:
print(vect.fit_transform(contents['content']).toarray())

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [ ]:
tfidf_df

,.,.kcb,.ktx,.거의,.국민신한하나산업토스카뱅케뱅우체국,.글,.남들,.생활비,.세전,.스이카,...,휴대폰요금,흑흑마일리지,흙수저,흙흙.,흡사한,힘드네,힘든,힘든데,힘들었고,힘듦더현대는
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
324,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
325,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
326,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
vect.vocabulary_

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

# initialize tf-idf transformer object
transformer = TfidfTransformer(smooth_idf=False)
# use counts from count vectorizer results to compute tf-idf values
tfidf = transformer.fit_transform(X)
# convert sparse matrix to numpy array to view
tfidf.toarray()

array([[0.08869108, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.04937479, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [ ]:
tfidf.toarray().sum()

1549.3800723783747

In [ ]:
column_sums = tfidf_df.sum()
columns_to_keep  = column_sums[column_sums > 5].index.tolist() # 5번 이상 언급
df_keep = tfidf_df[columns_to_keep]
df_keep

,.,1,10,100,100만원,10만원,1년에,2,20대,3,...,한도,할인,해서,해외,현대,현대카드,현아골,형님들,형들,혜택
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,0,0,2,0,0,0,0,0,0,0,...,0,2,1,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
324,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
325,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
326,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

# initialize tf-idf transformer object
transformer = TfidfTransformer(smooth_idf=False)
# use counts from count vectorizer results to compute tf-idf values
tfidf = transformer.fit_transform(df_keep)
# convert sparse matrix to numpy array to view
tfidf.toarray()

array([[0.24453508, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.10900944, 0.        , 0.19157237, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.2262721 , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [ ]:
df_keep = pd.DataFrame(tfidf.toarray(), columns = columns_to_keep)
df_keep

,.,1,10,100,100만원,10만원,1년에,2,20대,3,...,한도,할인,해서,해외,현대,현대카드,현아골,형님들,형들,혜택
0,0.244535,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.109009,0.0,0.191572,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.175093,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.0,0.226272,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.206808,0.103404,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.0,0.000000,0.0,0.0,0.0,0.295461,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
324,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
325,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.628035,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
326,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
df_keep.to_csv("/content/gdrive/MyDrive/ITStudy/크롤링/df_keep.csv", index=False)

#n=300

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [ ]:
for idx, row in contents.iterrows():
    clean_content = clean_str(row.content)
    clean_reply = clean_str(row.reply)
    contents.at[idx,'content'] = clean_content
    contents.at[idx, 'reply'] = clean_reply

In [ ]:
import pandas as pd


# 크롤링 결과 저장한 것 불러오기
contents = pd.read_csv('/content/gdrive/MyDrive/ITStudy/크롤링/contents.csv')
contents['content'] = contents['title'] + contents['contents']
contents = contents[['id','content']]
contents['reply'] = ''
reply = pd.read_csv('/content/gdrive/MyDrive/ITStudy/크롤링/reply.csv',encoding='utf-8')

for idx, row in contents.iterrows():
    reply_arr = ""
    for didx, drow in reply.iterrows():
        if row.id == drow.id:
            reply_arr += drow.reply_content
    contents.at[idx, 'reply'] = reply_arr

In [ ]:
contents.head()

,id,content,reply
0,308999,우리카드 추천해주실분계신가요?\n실적없고 그냥 국내 할인혜택 한번씩 쓸까해서 체...,실적 없는거면 카드의정석 에브리원 어떰? - dc App정보보겠씸 ㄳㄳ신카 실적없...
1,308946,"아 진짜 너무 고민중인데 카드 한 장만 추천해줘라\n매 월 꿀카드, 톡마포, 더 나...",+ 매달 스포티비 때문에 네이버페이 쓰는데 신한 네이버페이 알아볼까?상품권 6% 전...
2,308883,알뜰 교통카드에 관해서 문의 좀....!!\n0. 현재예전에 카드사에서 추천해줘서내...,알뜰교통에 신용은 연회비가 아까운데 체크로 ㄱ 실사할려면 우리 체크가 좋고 실사는 ...
3,308843,국민에서 카드 추천 연락 왔는데\n조건 충족해서 전화 온건가…? 내가 소득 없는 학...,마케팅 동의 철회 ㄱ
4,308815,조언 부탁드립니다!\n제가 1년에 한번씩은 꼭 해외여행을 가서 마일리지 카드를쓰기도...,마일리지 카드로 상테크 돌리고 평소엔 할인적립카드 쓰셈네 이게 제일 좋겠네요 감사합...


In [ ]:
# 한글만 남기고 특수문자, 숫자, 한자 등 제거
import re
p = re.compile('[가-힇ㄱ-ㅎㅏ-ㅣ]*')
for idx, content in enumerate(contents['content']):
    res = ' '.join(p.findall(contents['content'][idx]))
    contents['content'][idx] = ' '.join(res.split())

<ipython-input-315-bfa499876065>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  contents['content'][idx] = ' '.join(res.split())


In [ ]:
# Headline 형태소 분석 (명사, 동사, 형용사)
from konlpy.tag import Okt
def extractKeywords(okt,text):
    noun = []
    verb = []
    adj = []
    for word, pos in okt.pos(text, stem=True):
        if pos == 'Noun':
            noun.append(word)
        elif pos == 'Verb':
            verb.append(word)
        elif pos == 'Adjective':
            adj.append(word)
    return noun, verb, adj


okt = Okt()
df_contents = pd.DataFrame(None, columns=['noun','verb','adj','all'])
df_contents.loc[0] = [1,2,3,4]
for idx,text in enumerate(contents['content']):
    n,v,a = extractKeywords(okt,text)
    df_contents.loc[idx] = [' '.join(n),' '.join(v),' '.join(a), ' '.join(n)+' '+' ']

In [ ]:
# 앞서 크롤링 했던 DataFrame이랑 합치기 (합쳐놓는게 나중에 편할 것 같다고 판단하였음)
df_contents = pd.concat([contents,df_contents], axis=1)


# df_final['all']에 TF-IDF 적용
from sklearn.feature_extraction.text import TfidfVectorizer
import  numpy  as np

tfidf = TfidfVectorizer()
res = tfidf.fit_transform(df_contents['all']).toarray()
print(res.shape)

(328, 1646)


In [ ]:
df_contents

,id,content,reply,noun,verb,adj,all
0,308999,우리카드 추천해주실분계신가요 실적없고 그냥 국내 할인혜택 한번씩 쓸까해서 체크도 좋...,실적 없는거면 카드의정석 에브리원 어떰? - dc App정보보겠씸 ㄳㄳ신카 실적없...,우리카드 추천 분계 신가 실적 그냥 국내 할인 혜택 한번 체크 뭐 곘 하나카드 라운...,해주다 쓸다 하다 모르다 하다 모르다,없다 좋다 좋다,우리카드 추천 분계 신가 실적 그냥 국내 할인 혜택 한번 체크 뭐 곘 하나카드 라운...
1,308946,아 진짜 너무 고민중인데 카드 한 장만 추천해줘라 매 월 꿀카드 톡마포 더 나은 나...,+ 매달 스포티비 때문에 네이버페이 쓰는데 신한 네이버페이 알아볼까?상품권 6% 전...,진짜 고민 카드 장만 추천 매 월 꿀 카드 톡 마포 더 나은 나사 카 상 테크 달 ...,하다 해주다 쓴다 하다 하다 하다 되다 하다 하다 하다 하다 가다 가다,그렇다 있다 같다,진짜 고민 카드 장만 추천 매 월 꿀 카드 톡 마포 더 나은 나사 카 상 테크 달 ...
2,308883,알뜰 교통카드에 관해서 문의 좀 현재예전에 카드사에서 추천해줘서내가 지금 국민 탄탄...,알뜰교통에 신용은 연회비가 아까운데 체크로 ㄱ 실사할려면 우리 체크가 좋고 실사는 ...,알뜰 교통카드 관해 문의 좀 현재 예전 카드 사 추천 내 지금 국민 탄탄 퍼플 팩 ...,해주다 쓰다 채우다 받다 쓸다 보다 쓴다 치다 하다 하다 되다 쓴다 보다 쓰다 하다...,있다 있다 없다 없다 없다 좋다 번하다 같다 그렇다 좋다 없다 아니다 있다 있다 있다,알뜰 교통카드 관해 문의 좀 현재 예전 카드 사 추천 내 지금 국민 탄탄 퍼플 팩 ...
3,308843,국민에서 카드 추천 연락 왔는데 조건 충족해서 전화 온건가 내가 소득 없는 학식이라...,마케팅 동의 철회 ㄱ,국민 카드 추천 연락 조건 충족 전화 온건 내 소득 학식 전화 마무리,오다 하다 하다 하다,없다,국민 카드 추천 연락 조건 충족 전화 온건 내 소득 학식 전화 마무리
4,308815,조언 부탁드립니다 제가 년에 한번씩은 꼭 해외여행을 가서 마일리지 카드를쓰기도 하고...,마일리지 카드로 상테크 돌리고 평소엔 할인적립카드 쓰셈네 이게 제일 좋겠네요 감사합...,조언 제 년 한번 꼭 해외여행 마일리지 카드 쓰기 상 테크 장 이번 년 특 변수 마...,가다 하다 돌리다 생기다 하다 들다 가다 쓸다 쓸다 되다,부탁드리다 있다 바쁘다 있다 아니다 부탁드리다,조언 제 년 한번 꼭 해외여행 마일리지 카드 쓰기 상 테크 장 이번 년 특 변수 마...
...,...,...,...,...,...,...,...
323,299135,병원비결제 카드 추천 병원비 결제할 일이 있이 생겼는데 정도 내가 미리미리 신한 레...,그니마 카정포 여러개 인데 전월실적 필요하고 에블라도 돈 적게 해줘서 대체체 없음...,병원 비 결제 카드 추천 병원 비 결제 정도 내 미리 미리 신한 레이디 클래식 단종...,하다 생기다 되다 받다 찾아보다 알다,있다 없다 좋다 있다,병원 비 결제 카드 추천 병원 비 결제 정도 내 미리 미리 신한 레이디 클래식 단종...
324,299128,월 쓰는데 카드추천좀 원큐쿠폰어쩌고 카드 매달 정도 카드값나감혜택 뭐가 있는지 받고...,"하포신한 하이포인트ㄱㅅ하포실적 150채우고, 원카드 쓰고 싶으면 하포가 젤 편하고 ...",월 카드 추천 좀 큐 쿠폰 쩌 카드 매달 정도 카드 값 혜택 뭐 거도 모름 그냥 거...,쓰다 나가다 받다 쓰다 쓰다 나가다 살다 쓴다,있다 있다 없다 이렇다,월 카드 추천 좀 큐 쿠폰 쩌 카드 매달 정도 카드 값 혜택 뭐 거도 모름 그냥 거...
325,299122,실사용으로 쓰는데 신용카드 추천좀 현재 통신사 에 할인 톡마포 나사카 쓰는데,이미 잘쓰고있구만 거기서 뭐 추가할거없음 카정포 30써야 1만원인데 통신사가 2천원...,실 사용 신용카드 추천 좀 현재 통신사 할인 톡 마포 나사 카,쓰다 쓰다,,실 사용 신용카드 추천 좀 현재 통신사 할인 톡 마포 나사 카
326,299121,대학생 카드 추천좀 전역하고 군적금 다 부모님 드림 소득 원이고 한달마다 용돈 받아...,ㄴㄴ 걍 나사카 계속 쓰셈삼성페이 등록 안돼서 불편했는데 걍 써야겠네 - dc A...,대학생 카드 추천 좀 전역 군 적금 부모님 드림 소득 원 달마 용돈 잔고 가지 카드...,받다 써다 쓰다 하다,없다 있다 있다,대학생 카드 추천 좀 전역 군 적금 부모님 드림 소득 원 달마 용돈 잔고 가지 카드...


In [ ]:
# TF-IDF 기준 상위 n_top 개수 키워드 추출
n_top = 100
importance = np.argsort(np.asarray(res.sum(axis=0)).ravel())[::-1]
tfidf_feature_names_out = np.array(tfidf.get_feature_names_out())
print(tfidf_feature_names_out[importance[:n_top]])

['카드' '추천' '정도' '만원' '사용' '신용카드' '결제' '하나' '신한' '할인' '현대' '해외' '혜택' '발급'
 '국민' '실적' '드림' '적립' '지금' '마포' '네이버' '포인트' '페이' '이상' '통신비' '소비' '그냥' '다른'
 '테크' '쿠팡' '한도' '신한카드' '삼성' '체크카드' '쇼핑' '국민카드' '할부' '원카드' '무지' '롯데' '온라인'
 '학식' '우리' '생각' '현재' '회비' '제로' '편의점' '거의' '교통비' '아이폰' '이용' '일본' '주로' '알뜰'
 '현아' '초년' '마일리지' '통신사' '고민' '사람' '부탁' '쓰기' '자주' '형님' '교통' '식비' '캐시' '사회'
 '디자인' '제일' '공과금' '한번' '그린' '라운지' '배달' '클래식' '부모님' '적금' '은행' '프로' '와우'
 '메인' '하이' '게임' '아멕스' '평균' '신용' '순서' '이미지' '개월' '단종' '나머지' '가온' '예정' '케이'
 '관리' '상품권' '직구' '고정']


In [ ]:
# TF-IDF 기준 상위 top 개수만큼 품사별 키워드 추출 (명사, 동사, 형용사)
def topKeywords(okt,text):
    noun = []
    verb = []
    adj = []
    for word, pos in okt.pos(text, stem=True):
        if pos == 'Noun':
            noun.append(word)
        elif pos == 'Verb':
            verb.append(word)
        elif pos == 'Adjective':
            adj.append(word)
    return noun, verb, adj


noun,verb,adj = topKeywords(okt,' '.join(tfidf_feature_names_out[importance[:n_top]]))


noun_top = 100
print(f'명사 top : {noun[:noun_top]}')
verb_top = 10
print(f'동사 top : {verb[:verb_top]}')
adj_top = 10
print(f'형용사 top : {adj[:adj_top]}')

명사 top : ['카드', '추천', '정도', '만원', '사용', '신용카드', '결제', '하나', '신한', '할인', '현대', '해외', '혜택', '발급', '국민', '실적', '드림', '적립', '지금', '마포', '네이버', '포인트', '페이', '이상', '통신비', '소비', '그냥', '다른', '테크', '쿠팡', '한도', '신한카드', '삼성', '체크카드', '쇼핑', '국민카드', '할부', '원카드', '무지', '롯데', '온라인', '학식', '우리', '생각', '현재', '회비', '제로', '편의점', '거의', '교통비', '아이폰', '이용', '일본', '주로', '알뜰', '현아', '초년', '마일리지', '통신사', '고민', '사람', '부탁', '쓰기', '자주', '형님', '교통', '식비', '캐시', '사회', '디자인', '제일', '공과금', '한번', '그린', '라운지', '배달', '클래식', '부모님', '적금', '은행', '프로', '와우', '메인', '하이', '게임', '아멕스', '평균', '신용', '순서', '이미지', '개월', '단종', '나머지', '가온', '예정', '케이', '관리', '상품권', '직구', '고정']
동사 top : []
형용사 top : []
